# Matching Load and Generation to the Networks
First I will do 1 case to see how the workflow is



In [ ]:
"""
Setup environment and load the base PyPSA-Earth network for a specified country.
"""

import os
import sys
import warnings
import pypsa
import warnings
import pypsa
from pathlib import Path
import os
import copy
from os.path import join
import pandas as pd
import geopandas as gpd

# Import all dirs
parent_dir = Path(os.getcwd()).parents[0]
sys.path.append(str(parent_dir))
from src.paths import all_dirs

dirs = all_dirs()

import logging
from pathlib import Path
# Add PyPSA-Earth scripts to the system path
scripts_path = os.path.join(parent_dir.parents[0], "pypsa-earth", "scripts")
print(scripts_path)
assert os.path.isdir(scripts_path), f"Path not found: {scripts_path}"

sys.path.append(scripts_path)


c:\Repositories\Repos\pypsa-earth-project\pypsa-earth\scripts


In [24]:


LOG_FILE = join(parent_dir, "logs.log")

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s: %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
    handlers=[logging.FileHandler(LOG_FILE, encoding="utf-8")],
)


# Suppress warnings
warnings.simplefilter("ignore", category=FutureWarning)
warnings.simplefilter("ignore", category=UserWarning)
# Suppress unnecessary warnings for cleaner output
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

# Load the base network file path
network_dir = dirs["data/processed/networks"]
network_files = [
    "network_original",
    "network_snapped",
    "network_expanded",
    "network_expanded_no_orphans",
    "network_nuclear",
    "network_prod_mix",
    "network_base",
]

networks_dict = {
    nf: pypsa.Network(join(network_dir, f"{nf}.nc")) for nf in network_files
}


C:\Users\fpenaherrera_vaca\AppData\Local\Temp\ipykernel_4644\157704434.py:3: ResourceWarning:

unclosed file <_io.TextIOWrapper name='c:\\Repositories\\Repos\\pypsa-earth-project\\EcuadorElectricGrid\\logs.log' mode='a' encoding='utf-8'>



In [25]:
#base network
network = networks_dict["network_base"].copy()

In [26]:
# loads
# For the base case, I take the 2025 trend scenario with linear growt
path_loads = dirs["data/processed/scaled_loads"]
load_profile_name = "load_base_2030_linear.csv"
load_profile_file = join(path_loads, load_profile_name)

load_profile = pd.read_csv(load_profile_file, index_col=0, parse_dates=True)
# Add the load profile to the network
network.madd(
    "Load", load_profile.columns, bus=load_profile.columns, p_set=load_profile
)
#network_base.loads_t.p_set
# Check peak load (rounded to 1 decimal place)
peak_load = round(network.loads_t.p_set.T.sum().max() / 1000, 1)
logging.info(f"Peak load: {peak_load} GW")  # GW

In [27]:
ppl_file = join(dirs["data/processed/generation"],
                "powerplants_all.csv")

In [28]:
ppl = pd.read_csv(ppl_file, index_col=0)

In [29]:
# Update ppl to match pypsa network
import powerplantmatching as pm
# Load config file
ppmatching = os.path.join(
    Path(os.getcwd()).parents[1],
      "pypsa-earth", "configs", "powerplantmatching_config.yaml"
)
print(ppmatching)
config = pm.get_config(ppmatching)

# Select target countries
config["target_countries"] = ["EC"]



c:\Repositories\Repos\pypsa-earth-project\pypsa-earth\configs\powerplantmatching_config.yaml


In [32]:
ppl.columns

Index([' Fueltype      ', ' Technology  ', ' Set           ', ' Country',
       ' Capacity ', ' Efficiency', ' Duration', ' Volume_Mm3', ' DamHeight_m',
       ' StorageCapacity_MWh', ' DateIn', ' DateRetrofit', ' DateOut',
       ' lat       ', ' lon        ', ' EIC', ' projectID'],
      dtype='object')

In [30]:


ppl_f = ppl.powerplant.fill_missing_commissioning_years()
ppl_p = ppl_f.powerplant.to_pypsa_names()
ppl_p = ppl_p.dropna(axis=1, how="all")



AttributeError: 'DataFrame' object has no attribute 'DateIn'

In [ ]:
# Filter Power Plants by size and by conection to network
FILTERING_POWER = 10
ppl_connected = ppl_p[ppl_p['Set']=="S.N.I."]  # Keep only plants connected to the network
ppl_connected_current = ppl_connected[ppl_connected['DateIn']<=2017]  # Example: filter plants commissioned before or in 2024
#ppl_connected_future = ppl_connected[ppl_connected['DateIn']>2017]  # Example: filter plants commissioned before or in 2024
ppl_filtered = ppl_connected[ppl_connected['p_nom']>=FILTERING_POWER]  # Example: filter plants with capacity >= 100 kW

total_capacity = ppl_p['p_nom'].sum()
total_connected = ppl_connected['p_nom'].sum()
total_connected_current = ppl_connected_current['p_nom'].sum()
total_filtered = ppl_filtered['p_nom'].sum()
logging.info(f"\nTotal capacity: {total_capacity} kW,\nConnected capacity: {total_connected} kW, \
             \nConnected current capacity: {total_connected_current} kW, \
             \nFiltered capacity: {total_filtered} kW")  

In [ ]:
import matplotlib.pyplot as plt

# Define Coordinate Reference Systems (CRS)
geo_crs = 4326  # general geographic projection, not used for metric measures. "EPSG:4326" is the standard used by OSM and google maps
area_crs = 6933 # projection for area measurements only. Possible recommended values are Global Mollweide "ESRI:54009" (but 54009 is not supported by atlite, use 6933 instead)
# distance_crs = 3857  # projection for distance measurements only. Possible recommended values are "EPSG:3857" (used by OSM and Google Maps) -> currently not used

# Read coordinates from powerplants and buses
ppl_geometry = gpd.points_from_xy(ppl_filtered['lon'], ppl_filtered['lat'])
buses_geometry = gpd.points_from_xy(network.buses['lon'], network.buses['lat'])
#ppl_capacities =gpd.GeoDataFrame(ppl_filtered, geometry=ppl_geometry, crs=geo_crs)
#buses_capacities =gpd.GeoDataFrame(network.buses, geometry=buses_geometry, crs=geo_crs)

In [ ]:
from scipy.spatial import cKDTree as KDTree
import numpy as np

substation_i = network.buses.query("substation_lv").index
kdtree = KDTree(network.buses.loc[substation_i, ["x", "y"]].values)
ppl_i = ppl_filtered.index

tree_i = kdtree.query(ppl_filtered.loc[ppl_i, ["lon", "lat"]].values)[1]
ppl_filtered.loc[ppl_i, "bus"] = substation_i.append(pd.Index([np.nan]))[tree_i]

In [ ]:
network.madd("Generator", 
             ppl_filtered.index, bus=ppl_filtered.bus,
               p_nom=ppl_filtered.p_nom, carrier=ppl_filtered.carrier)

network.generators